# [Exploration 15] Transformer 사용하여 한국어 챗봇 만들기 

이번 프로젝트는 한국어 데이터셋을 가지고 Transformer 개념을 도입한 챗봇을 만들어 본다.   
Transformer는 RNN, LSTM, seq2seq, Attention 메커니즘까지 발전해오며 자연어 처리에 대한 모든 이해를 포함하고 있는 개념이라 생각한다. 그런만큼 개념적으로 선행해야할 부분과 이해해야할 부분이 많아 개인적으로는 제일 어려웠던 프로젝트였다.   
개념적인 부분을 전체적으로 이해하는데 시간이 더 필요했다. 현재까지 공부한 내용으로는 아직 각각의 클래스와 함수가 어떤식으로 작동하는지 이해가 되지 않았다.   
이번 프로젝트 제출은 전체적인 흐름과 과정을 설명하는데 목적을 두고, 디테일한 이해는 시간을 가지고 차차 이해해보도록 하겠다.

## Step 1. 데이터 수집하기

`$ wget https://github.com/songys/Chatbot_data/raw/master/ChatbotData%20.csv`   
한국어 데이터셋은 송영숙님이 공개한 챗봇 데이터를 사용한다.   
   
   
이 데이터는 누군가에게 위로가 되는 대답을 해줄 수 있는 모델을 만드는 데 목적이 있었다고 한다. 실제로 레이블값을 보면 0 (일상), 1(이별, 부정), 2(사랑 긍정)로 레이블링 되어있다.

In [30]:
import tensorflow as tf
import tensorflow_datasets as tfds
import os
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

필요한 모듈을 import 한다.

In [31]:
file_path = os.getenv("HOME") + '/aiffel/songys_chatbot/ChatbotData .csv'
data = pd.read_csv(file_path)

In [32]:
data.head(5)

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


label값은 사용하지 않으므로 제거하도록 하겠다.

In [36]:
data = data.drop(['label'], axis=1)

## Step 2. 데이터 전처리하기

결측치와 정규화, 불용어를 제거해준다.   
다만, 중복값에 대해서는 같은 질문에 다른 대답, 다른 질문에 같은 대답의 경우가 있을수도 있으므로 중복값은 제거하지 않도록 한다.

In [38]:
input = data['Q']
output = data['A']

일단 테스트삼아 전처리를 하지않고 진행해보도록 하겠다.

## Step 3. SubwordTextEncoder 사용하기

In [42]:
tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(input + output, target_vocab_size=2**13)

In [43]:
# 시작 토큰과 종료 토큰에 고유한 정수를 부여합니다.
START_TOKEN, END_TOKEN = [tokenizer.vocab_size], [tokenizer.vocab_size + 1]
print("슝=3")

슝=3


In [44]:
print('START_TOKEN의 번호 :' ,[tokenizer.vocab_size])
print('END_TOKEN의 번호 :' ,[tokenizer.vocab_size + 1])

START_TOKEN의 번호 : [8361]
END_TOKEN의 번호 : [8362]


In [45]:
VOCAB_SIZE = tokenizer.vocab_size + 2
print(VOCAB_SIZE)

8363


In [47]:
print('정수 인코딩 후의 21번째 질문 샘플: {}'.format(tokenizer.encode(input[21])))
print('정수 인코딩 후의 21번째 답변 샘플: {}'.format(tokenizer.encode(output[21])))



정수 인코딩 후의 21번째 질문 샘플: [5824, 602, 2498, 4170]
정수 인코딩 후의 21번째 답변 샘플: [2683, 7666, 6, 6375, 92, 8151]


In [48]:
MAX_LENGTH = 40

In [49]:
def tokenize_and_filter(inputs, outputs):
  tokenized_inputs, tokenized_outputs = [], []
  
  for (sentence1, sentence2) in zip(inputs, outputs):
    # 정수 인코딩 과정에서 시작 토큰과 종료 토큰을 추가
    sentence1 = START_TOKEN + tokenizer.encode(sentence1) + END_TOKEN
    sentence2 = START_TOKEN + tokenizer.encode(sentence2) + END_TOKEN

    # 최대 길이 40 이하인 경우에만 데이터셋으로 허용
    if len(sentence1) <= MAX_LENGTH and len(sentence2) <= MAX_LENGTH:
      tokenized_inputs.append(sentence1)
      tokenized_outputs.append(sentence2)
  
  # 최대 길이 40으로 모든 데이터셋을 패딩
  tokenized_inputs = tf.keras.preprocessing.sequence.pad_sequences(
      tokenized_inputs, maxlen=MAX_LENGTH, padding='post')
  tokenized_outputs = tf.keras.preprocessing.sequence.pad_sequences(
      tokenized_outputs, maxlen=MAX_LENGTH, padding='post')
  
  return tokenized_inputs, tokenized_outputs

In [51]:
questions, answers = tokenize_and_filter(input, output)
print('단어장의 크기 :',(VOCAB_SIZE))
print('필터링 후의 질문 샘플 개수: {}'.format(len(questions)))
print('필터링 후의 답변 샘플 개수: {}'.format(len(answers)))

단어장의 크기 : 8363
필터링 후의 질문 샘플 개수: 11823
필터링 후의 답변 샘플 개수: 11823


Maxlen을 노드에서 그대로 가져온 40개로 설정하였는데 40자가 넘는 문장이 없었던것 같다.   
결과물을 보고 조정해보도록 하겠다.

In [52]:
BATCH_SIZE = 64
BUFFER_SIZE = 20000

# 디코더는 이전의 target을 다음의 input으로 사용합니다.
# 이에 따라 outputs에서는 START_TOKEN을 제거하겠습니다.
dataset = tf.data.Dataset.from_tensor_slices((
    {
        'inputs': questions,
        'dec_inputs': answers[:, :-1]
    },
    {
        'outputs': answers[:, 1:]
    },
))

dataset = dataset.cache()
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

In [53]:
def transformer(vocab_size,
                num_layers,
                units,
                d_model,
                num_heads,
                dropout,
                name="transformer"):
  inputs = tf.keras.Input(shape=(None,), name="inputs")
  dec_inputs = tf.keras.Input(shape=(None,), name="dec_inputs")

	# 인코더에서 패딩을 위한 마스크
  enc_padding_mask = tf.keras.layers.Lambda(
      create_padding_mask, output_shape=(1, 1, None),
      name='enc_padding_mask')(inputs)

  # 디코더에서 미래의 토큰을 마스크 하기 위해서 사용합니다.
  # 내부적으로 패딩 마스크도 포함되어져 있습니다.
  look_ahead_mask = tf.keras.layers.Lambda(
      create_look_ahead_mask,
      output_shape=(1, None, None),
      name='look_ahead_mask')(dec_inputs)

  # 두 번째 어텐션 블록에서 인코더의 벡터들을 마스킹
  # 디코더에서 패딩을 위한 마스크
  dec_padding_mask = tf.keras.layers.Lambda(
      create_padding_mask, output_shape=(1, 1, None),
      name='dec_padding_mask')(inputs)

  # 인코더
  enc_outputs = encoder(
      vocab_size=vocab_size,
      num_layers=num_layers,
      units=units,
      d_model=d_model,
      num_heads=num_heads,
      dropout=dropout,
  )(inputs=[inputs, enc_padding_mask])

  # 디코더
  dec_outputs = decoder(
      vocab_size=vocab_size,
      num_layers=num_layers,
      units=units,
      d_model=d_model,
      num_heads=num_heads,
      dropout=dropout,
  )(inputs=[dec_inputs, enc_outputs, look_ahead_mask, dec_padding_mask])

  # 완전연결층
  outputs = tf.keras.layers.Dense(units=vocab_size, name="outputs")(dec_outputs)

  return tf.keras.Model(inputs=[inputs, dec_inputs], outputs=outputs, name=name)
print("슝=3")

슝=3


## Step 4. 모델 구성하기   
위 실습 내용을 참고하여 트랜스포머 모델을 구현합니다.

In [54]:
tf.keras.backend.clear_session()

# 하이퍼파라미터
NUM_LAYERS = 2 # 인코더와 디코더의 층의 개수
D_MODEL = 256 # 인코더와 디코더 내부의 입, 출력의 고정 차원
NUM_HEADS = 8 # 멀티 헤드 어텐션에서의 헤드 수 
UNITS = 512 # 피드 포워드 신경망의 은닉층의 크기
DROPOUT = 0.1 # 드롭아웃의 비율

model = transformer(
    vocab_size=VOCAB_SIZE,
    num_layers=NUM_LAYERS,
    units=UNITS,
    d_model=D_MODEL,
    num_heads=NUM_HEADS,
    dropout=DROPOUT)

model.summary()

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Model: "transformer"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inputs (InputLayer)             [(None, None)]       0                                            
__________________________________________________________________________________________________
dec_inputs (InputLayer)         [(None, None)]       0                                            
__________________________________________________________________________________________________
enc_padding_mask (Lambda)       (None, 1, 1, None)   0           inputs[0][0]        

In [55]:
def loss_function(y_true, y_pred):
  y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))
  
  loss = tf.keras.losses.SparseCategoricalCrossentropy(
      from_logits=True, reduction='none')(y_true, y_pred)

  mask = tf.cast(tf.not_equal(y_true, 0), tf.float32)
  loss = tf.multiply(loss, mask)

  return tf.reduce_mean(loss)

In [56]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):

  def __init__(self, d_model, warmup_steps=4000):
    super(CustomSchedule, self).__init__()

    self.d_model = d_model
    self.d_model = tf.cast(self.d_model, tf.float32)

    self.warmup_steps = warmup_steps

  def __call__(self, step):
    arg1 = tf.math.rsqrt(step)
    arg2 = step * (self.warmup_steps**-1.5)

    return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [58]:
learning_rate = CustomSchedule(D_MODEL)

optimizer = tf.keras.optimizers.Adam(
    learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

def accuracy(y_true, y_pred):
  y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))
  return tf.keras.metrics.sparse_categorical_accuracy(y_true, y_pred)

model.compile(optimizer=optimizer, loss=loss_function, metrics=[accuracy])

In [60]:
EPOCHS = 20
model.fit(dataset, epochs=EPOCHS, verbose=1)

Epoch 1/20
185/185 [==============================] - 12s 63ms/step - loss: 0.9731 - accuracy: 0.0522
Epoch 2/20
185/185 [==============================] - 12s 63ms/step - loss: 0.9070 - accuracy: 0.0559
Epoch 3/20
185/185 [==============================] - 12s 63ms/step - loss: 0.8489 - accuracy: 0.0592
Epoch 4/20
185/185 [==============================] - 12s 63ms/step - loss: 0.7869 - accuracy: 0.0639
Epoch 5/20
185/185 [==============================] - 12s 63ms/step - loss: 0.7195 - accuracy: 0.0704
Epoch 6/20
185/185 [==============================] - 12s 64ms/step - loss: 0.6443 - accuracy: 0.0786
Epoch 7/20
185/185 [==============================] - 12s 63ms/step - loss: 0.5636 - accuracy: 0.0876
Epoch 8/20
185/185 [==============================] - 12s 63ms/step - loss: 0.4812 - accuracy: 0.0971
Epoch 9/20
185/185 [==============================] - 12s 63ms/step - loss: 0.3987 - accuracy: 0.1077
Epoch 10/20
185/185 [==============================] - 12s 63ms/step - loss: 0.319

## Step 5. 모델 평가하기   
Step 1에서 선택한 전처리 방법을 고려하여 입력된 문장에 대해서 대답을 얻는 예측 함수를 만듭니다.

In [64]:
def preprocess_sentence(sentence):
  sentence = sentence.lower().strip()

  # 단어와 구두점(punctuation) 사이의 거리를 만듭니다.

  sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
  sentence = re.sub(r'[" "]+', " ", sentence)

  # (a-z, A-Z, ".", "?", "!", ",")를 제외한 모든 문자를 공백인 ' '로 대체합니다.
  sentence = re.sub(r"[^a-zA-Z?.!,]+", " ", sentence)
  sentence = sentence.strip()
  return sentence

In [65]:
def decoder_inference(sentence):
  sentence = preprocess_sentence(sentence)

  # 입력된 문장을 정수 인코딩 후, 시작 토큰과 종료 토큰을 앞뒤로 추가.
  # ex) Where have you been? → [[8331   86   30    5 1059    7 8332]]
  sentence = tf.expand_dims(
      START_TOKEN + tokenizer.encode(sentence) + END_TOKEN, axis=0)

  # 디코더의 현재까지의 예측한 출력 시퀀스가 지속적으로 저장되는 변수.
  # 처음에는 예측한 내용이 없음으로 시작 토큰만 별도 저장. ex) 8331
  output_sequence = tf.expand_dims(START_TOKEN, 0)

  # 디코더의 인퍼런스 단계
  for i in range(MAX_LENGTH):
    # 디코더는 최대 MAX_LENGTH의 길이만큼 다음 단어 예측을 반복합니다.
    predictions = model(inputs=[sentence, output_sequence], training=False)
    predictions = predictions[:, -1:, :]

    # 현재 예측한 단어의 정수
    predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

    # 만약 현재 예측한 단어가 종료 토큰이라면 for문을 종료
    if tf.equal(predicted_id, END_TOKEN[0]):
      break

    # 예측한 단어들은 지속적으로 output_sequence에 추가됩니다.
    # 이 output_sequence는 다시 디코더의 입력이 됩니다.
    output_sequence = tf.concat([output_sequence, predicted_id], axis=-1)

  return tf.squeeze(output_sequence, axis=0)

In [66]:
def sentence_generation(sentence):
  # 입력 문장에 대해서 디코더를 동작 시켜 예측된 정수 시퀀스를 리턴받습니다.
  prediction = decoder_inference(sentence)

  # 정수 시퀀스를 다시 텍스트 시퀀스로 변환합니다.
  predicted_sentence = tokenizer.decode(
      [i for i in prediction if i < tokenizer.vocab_size])

  print('입력 : {}'.format(sentence))
  print('출력 : {}'.format(predicted_sentence))

  return predicted_sentence

In [67]:
sentence_generation('오늘 너무 피곤하다')

입력 : 오늘 너무 피곤하다
출력 : 지금 무슨 말을 해도 와닿지 않겠지만 잘할 수 있을 거예요.


'지금 무슨 말을 해도 와닿지 않겠지만 잘할 수 있을 거예요.'

In [69]:
sentence_generation('오늘 뭐해?')

입력 : 오늘 뭐해?
출력 : 저는 오래 살고 싶어요.


'저는 오래 살고 싶어요.'

# 회고:  
## 1. **이번 프로젝트에서 어려웠던 점**   
- 트랜스포머 모델에 대해 전체적으로 이해하지 못했다.   
- 각 파트에서 진행되는 코드에 대한 디테일한 사용을 이해하지 못했다.
   
   
## 2. **프로젝트를 진행하면서 알아낸 점 혹은 아직 모호한 점**   
- seq2seq의 어텐션 메커니즘을 넘어 트랜스포머라는 새로운 방식을 배울 수 있었다

## 3. **루브릭 평가 지표를 맞추기 위해 시도한 것들**   
1. 간단한 공백, 특수문자 처리, 토크나이징 등을 사용하였다.   
2. accuracy가 낮게 나왔는데 loss값은 안정화된것 같다. (어떻게 해석해야할지 모르겠다)   
3. 그럴듯한 한국어 답변이 나왔다.


## 4. **자기 다짐**   
이번 프로젝트는 사실 거의 제대로 진행하지 못했다. 구조에 대해 전체적으로 이해하지 못했고 (특히 트랜스포머 구조) 시간이 부족하여 전처리도 진행하지 못하고 제출을 한다. 하지만 트랜스포머는 이미지 분야로 비교하자면 CNN으로 이해될 만큼 중요한 내용이기 때문에 꼭 다시 학습을 하고 프로젝트를 다시 진행해봐야겠다.
   